In [1]:
import pandas as pd
import numpy as np
import sqlite3
import zipfile
import ast

## List of Dataframes:

In [2]:
# Dataframes
movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t', compression='gzip')
movie_info_drop = ['box_office', 'currency', 'dvd_date', 'studio']
movie_info_df = movie_info_df.drop(columns=movie_info_drop)
print(f'movie_info_df columns: \n{list(movie_info_df.columns)}\n')


movie_gross_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
print(f'movie_gross_df columns: \n{list(movie_gross_df.columns)}\n')

movie_budget_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
print(f'movie_budget_df columns: \n{list(movie_budget_df.columns)}\n')

movie_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', compression='gzip', encoding='latin-1')
movie_review_drop = ['rating']
movie_reviews_df = movie_reviews_df.drop(columns=movie_review_drop) # Dropping rating: poor formatting & lacking 50% of data.
print(f'movie_reviews_df columns: \n{list(movie_reviews_df.columns)}\n')


tmbd_movie_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
print(f'tmbd_movie_df columns: \n{list(tmbd_movie_df.columns)}\n')

movie_info_df columns: 
['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'runtime']

movie_gross_df columns: 
['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']

movie_budget_df columns: 
['id', 'release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']

movie_reviews_df columns: 
['id', 'review', 'fresh', 'critic', 'top_critic', 'publisher', 'date']

tmbd_movie_df columns: 
['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count']



**Combined Datraframe**: MAIN_DF

In [3]:
#change budget column name to title instead of movie
movie_budget_df = movie_budget_df.rename(columns={'movie': 'title'})
# Merge tmbd_movie_df and movie_gross_df]
main_df = pd.merge(tmbd_movie_df, movie_gross_df, on='title', how='left')
# Keep only the 'title' and 'production_budget' columns from movie_budget_df
main_budget_df = movie_budget_df[['title', 'production_budget']]
# Merge master_df and movie_budget_df
main_df = pd.merge(main_df, movie_budget_df, on='title', how='left')
# Drop unused column
column_to_drop = ['Unnamed: 0', 'release_date_y', 'domestic_gross_y', 'id_y', 'year']
main_df = main_df.drop(columns=column_to_drop)
#rename main_df columns to better reflact data
main_df = main_df.rename(columns={'id_x': 'id', 'release_date_x': 'release_date', 'domestic_gross_x': 'domestic_gross'})


Cleaning & filtering **MAIN_DF**:

In [5]:
# Filter to only english
main_df = main_df[main_df['original_language'] == 'en']
# Filter by 2010 and sooner
main_df = main_df[main_df['release_date'] >= '2010-01-01']
#Updateing 'production_budget' & 'worldwide_gross' columns
# First, replace $ and ,
main_df['production_budget'] = main_df['production_budget'].str.replace('$', '').str.replace(',', '')
main_df['worldwide_gross'] = main_df['worldwide_gross'].str.replace('$', '').str.replace(',', '')
# Fill NaNs with 0
main_df['production_budget'] = main_df['production_budget'].fillna(0)
main_df['worldwide_gross'] = main_df['worldwide_gross'].fillna(0)
# Now convert to int
main_df['production_budget'] = main_df['production_budget'].astype(int)
main_df['worldwide_gross'] = main_df['worldwide_gross'].astype(int)
# Adding Columns
main_df['net_profit'] = main_df['worldwide_gross'] - main_df['production_budget']
main_df['roi'] = main_df['net_profit'] / main_df['production_budget'] * 100
# Create a dictionary mapping genre IDs to names, all in lowercase
genre_dict = {28: 'action', 12: 'adventure', 16: 'animation', 35: 'comedy', 80: 'crime', 99: 'documentary', 
              18: 'drama', 10751: 'family', 14: 'fantasy', 36: 'history', 27: 'horror', 10402: 'music', 
              9648: 'mystery', 10749: 'romance', 878: 'science fiction', 10770: 'tv movie', 53: 'thriller', 
              10752: 'war', 37: 'western'}

def decode_genre_ids(genre_ids_string):
    # Convert string to list
    genre_ids = ast.literal_eval(genre_ids_string)
    # Return the genre names corresponding to the IDs
    return [genre_dict[id] for id in genre_ids if id in genre_dict]

# Apply the function to each value in the 'genre_ids' column
main_df['genres'] = main_df['genre_ids'].apply(decode_genre_ids)



In [6]:
main_df.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,studio,domestic_gross,foreign_gross,production_budget,worldwide_gross,net_profit,roi,genres
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,NaN,NaN,NaN,0,0,0,NaN,"[adventure, fantasy, family]"
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,P/DW,217600000.0,277300000,165000000,494870992,329870992,199.921813,"[fantasy, adventure, animation, family]"
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,Par.,312400000.0,311500000,170000000,621156389,451156389,265.386111,"[adventure, action, science fiction]"
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,WB,292600000.0,535700000,160000000,835524642,675524642,422.202901,"[action, science fiction, adventure]"
5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,NaN,NaN,NaN,95000000,223050874,128050874,134.790394,"[adventure, fantasy, family]"


In [7]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23184 entries, 0 to 26605
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          23184 non-null  object 
 1   id                 23184 non-null  int64  
 2   original_language  23184 non-null  object 
 3   original_title     23184 non-null  object 
 4   popularity         23184 non-null  float64
 5   release_date       23184 non-null  object 
 6   title              23184 non-null  object 
 7   vote_average       23184 non-null  float64
 8   vote_count         23184 non-null  int64  
 9   studio             2159 non-null   object 
 10  domestic_gross     2148 non-null   float64
 11  foreign_gross      1498 non-null   object 
 12  production_budget  23184 non-null  int64  
 13  worldwide_gross    23184 non-null  int64  
 14  net_profit         23184 non-null  int64  
 15  roi                2191 non-null   float64
 16  genres             231

In [8]:
main_df.isna().sum().sort_values(ascending=False)


foreign_gross        21686
domestic_gross       21036
studio               21025
roi                  20993
genres                   0
release_date             0
id                       0
original_language        0
original_title           0
popularity               0
vote_count               0
title                    0
vote_average             0
production_budget        0
worldwide_gross          0
net_profit               0
genre_ids                0
dtype: int64

### movie_info_df: cleaned

In [ ]:
movie_info_df.info()

In [ ]:
movie_info_df.isna().sum().sort_values(ascending=False)

Cleaning & Filtering for **MOVIE_INFO_DF**:

In [9]:
# Dropping 8 null rows with missing genre. Most info is missing with them:
movie_info_df = movie_info_df.dropna(subset=['genre'])

# filling all empty values with 'unknown' for ['synopsis', 'runtime', 'director', 'theater_date', 'writer']:
columns_to_fill = ['synopsis', 'runtime', 'director', 'theater_date', 'writer']
movie_info_df[columns_to_fill] = movie_info_df[columns_to_fill].fillna('unknown')

# Filtering date to 2010 and forward
movie_info_df['theater_date'] = pd.to_datetime(movie_info_df['theater_date'], format='%b %d, %Y', errors='coerce')
movie_info_df = movie_info_df[movie_info_df['theater_date'] >= 'Jan 1, 2010']



# changing runtime to int + removing 'minutes' + changing name of the column to reflect:
movie_info_df.loc[movie_info_df['runtime'] != 'unknown', 'runtime'] = movie_info_df.loc[movie_info_df['runtime'] != 'unknown', 'runtime'].str.replace(' minutes', '').astype(int)
movie_info_df = movie_info_df.rename(columns={'runtime': 'runtime_in_minutes'})

# drop all 26 unknown runtimes:
movie_info_df = movie_info_df[movie_info_df['runtime_in_minutes'] != 'unknown']


In [ ]:
movie_info_df.head(3)

**MOVIE_INFO_DF** Analysis:

In [ ]:
# 81% of the movie market is rated PG-13, R and NR:
movie_info_df['rating'].value_counts()

In [ ]:
# average movie length is just over 100 minutes
movie_info_df['runtime_in_minutes'].mean()

In [ ]:
# Top Directors
movie_info_df['director'].value_counts().head(20)

In [ ]:
# Top Writers
movie_info_df['writer'].value_counts().head(20)

In [ ]:
# Top Genres seem to be a mix of Drama & Comedy
movie_info_df['genre'].value_counts().head(20)

### movie_reviews_df: cleaned

In [ ]:
movie_reviews_df.info()

In [ ]:
movie_reviews_df.isna().sum().sort_values(ascending=False)

**MOVIE_REVIEWS_DF** Data cleaning:

In [ ]:
# Filtering to only top_critics == 1
movie_reviews_df = movie_reviews_df[movie_reviews_df['top_critic'] == 1]
# filling null 'publisher', 'critic', 'review' values with 'Unknown':
fill_columns = ['publisher', 'critic', 'review']
movie_reviews_df[fill_columns] = movie_reviews_df[fill_columns].fillna('unknown')
# Filtering date to 2010 and forward
movie_reviews_df['date'] = pd.to_datetime(movie_reviews_df['date'])
movie_reviews_df = movie_reviews_df[movie_reviews_df['date'] >= '2010-01-01']





In [ ]:
movie_reviews_df.head(20)

**MOVIE_REVIEW_DF** Analysis:

Top_Critics reviews = 4,713

In [ ]:
movie_reviews_df['top_critic'].sum()

Names of the top 20 critics:

In [ ]:
movie_reviews_df['critic'].value_counts().head(20)

Fresh vs. Rotten; 

- fresh = 60%
- Rotten = 40%

In [ ]:
movie_reviews_df['fresh'].value_counts()

---

# SQL DATABASE

* Unzipping the '**im.db.zip**' file 
* Connecting to database using '**conn**'
* Printing all of the **table names** within the database


**Important note**: movie_basics & movie_ratings are the most relevant per instructions

In [ ]:
# Unzip the database file
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('zippedData')

# Connect to the unzipped SQLite database
conn = sqlite3.connect('zippedData/im.db')

# Run test query
q = """
SELECT tbl_name AS table_name, sql
FROM sqlite_master 
WHERE type='table'
ORDER BY name;
"""
pd.read_sql(q, conn)

In [ ]:
# Movie Basics
q = '''
SELECT *
FROM movie_basics
WHERE original_title LIKE "Toy%"
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Moving Ratings
q = '''
SELECT *
FROM movie_ratings
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Directors
q = '''
SELECT *
FROM directors
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Known For
q = '''
SELECT *
FROM known_for
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Movie AKAs
q = '''
SELECT *
FROM movie_akas
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Persons
q = '''
SELECT *
FROM persons
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Pricipals
q = '''
SELECT *
FROM principals
LIMIT 3
;
'''
pd.read_sql(q, conn)

In [ ]:
# Writers
q = '''
SELECT *
FROM writers
LIMIT 3
;
'''
pd.read_sql(q, conn)

---

In [ ]:
# joined file
q = '''
SELECT *
FROM movie_basics mb
JOIN directors dr USING(movie_id)
JOIN principals pr USING(movie_id)
JOIN persons p USING(person_id)
JOIN movie_ratings mr USING(movie_id)
WHERE death_year IS NULL
GROUP BY movie_id
ORDER BY start_year
;
'''
pd.read_sql(q, conn)

---

# **Recommendations**

### Worldwide vs Domestic vs Foreign:

* based on the data from our dataframes, we have decided recommend launching your movie **Worldwide**. 
* We ran into a lot of data limitations when breaking down domestic vs foreign but we had plenty to work with as it pertained to **production budget** & **worldwide**.
* **We recommend launching the movie worldwide**

Further Filtering:

In [10]:
calc_df = main_df

# Filtering columns
calc_df = calc_df[calc_df['production_budget'] > 0]
calc_df = calc_df[calc_df['worldwide_gross'] > 0]
calc_df = calc_df[calc_df['roi'] > 0]
calc_df = calc_df[calc_df['domestic_gross'] > 0]
calc_df = calc_df[~calc_df['foreign_gross'].isna()]
# Sorting values by ROI
calc_df = calc_df.sort_values('roi', ascending=False)
# removing movie duplicates values in ROI
calc_df = calc_df.loc[~calc_df['roi'].duplicated(), :]

Based on **top 10 movies (ROI)**

* The best ROI ranges from **41,560% to 3,852%** increse
* **70%** of the top ROI movies are a mix of **Horror** and **Thriller**
* **We recommend making a HORROR/THRILLER MOVIE**

In [11]:
calc_df[['title', 'roi', 'genres']].head(10)

,title,roi,genres
14420,The Gallows,41556.474000,"[horror, thriller]"
5468,The Devil Inside,10075.949000,"[thriller, horror]"
2512,Insidious,6558.059067,"[horror, thriller]"
14490,Unfriended,6336.419800,"[horror, thriller]"
235,Paranormal Activity 2,5817.067733,"[horror, thriller]"
17455,Split,5479.296120,"[horror, thriller]"
1541,Get Out,5007.359020,"[comedy, animation]"
5253,Chernobyl Diaries,4141.172100,"[horror, thriller]"
2663,Paranormal Activity 3,4040.796880,[horror]
11133,Annabelle,3851.737231,[horror]


Here are the individual genre counts across all movies for visability:

In [12]:
genre_counts = calc_df['genres'].explode().value_counts()
genre_counts

drama              342
comedy             315
action             256
thriller           220
adventure          198
family             130
science fiction    122
crime              104
romance            104
fantasy            104
horror              94
animation           82
mystery             63
history             36
music               18
war                 17
documentary         11
western              7
Name: genres, dtype: int64

**Movie length (minutes)**

* We found that the average movie length was just over **106 mintues**.
* **We recommend keeping the movie length between 100-110 minutes**

In [13]:
# average movie length is just over 100 minutes
movie_info_df['runtime_in_minutes'].mean()

106.5228426395939

**Best DIRECTORS for the Horror/Thriller Genre**

* ENTER DETAILS HERE..... 
* **We recommend....**


In [21]:
# Enter SQL Code here


**Best WRITERS for the Horror/Thriller Genre**

* ENTER DETAILS HERE..... 
* **We recommend....**


In [ ]:
# Enter SQL Code here

**Best ACTORS for the Horror/Thriller Genre**

* ENTER DETAILS HERE..... 
* **We recommend....**


In [ ]:
# Enter SQL Code here